In [1]:
import os
from okcupid_stackoverflow.src.tf_idf.tf_idf import run_fit_tf_idf
from okcupid_stackoverflow.utils.git_utils import get_git_root

In [2]:
git_root_loc = get_git_root(os.getcwd())

In [ ]:
run_name = "akos_testing_pipeline_no_metadata"
use_metadata=False
run_fit_tf_idf(git_root_loc, run_id=run_name, save_external=True, use_metadata=use_metadata)

In [ ]:
run_name = "akos_testing_pipeline"
use_metadata=True
run_fit_tf_idf(git_root_loc, run_id=run_name, save_external=True, use_metadata=use_metadata)